In [ ]:
import scipy as sp
from scipy.sparse import spdiags
from scipy.sparse.linalg import spsolve
import numpy as np
import matplotlib.pyplot as plt

# Set up

In [ ]:
def create_matrix(x_space,alpha,beta,epsilon):
    n = len(x_space)-1
    h = x_space[1:]-x_space[:-1]
    #print h
    
    Phi = np.zeros_like(x_space)
    Phi[1:-1] = -.5*(h[:-1]+h[1:])
    Phi[0] = alpha
    Phi[-1] = beta
    
    main = np.ones_like(x_space)
    main[1:-1] = -epsilon/h[:-1]-epsilon/h[1:]
    
    upp_diag = np.zeros_like(x_space)
    upp_diag[2:] = epsilon/h[1:] - 1/2
    
    low_diag = np.zeros_like(x_space)
    low_diag[:-2] = epsilon/h[:-1] + 1/2
    
    A = spdiags([main, upp_diag, low_diag], [0,1,-1], n+1, n+1)
    
    return A, Phi

In [ ]:
def phi(x,i,linspace):
    h = linspace[1:] - linspace[:-1]
    if i == 0:
        if x >= linspace[i] and x <= linspace[i+1]:
            return (linspace[i+1]-x)/h[i]
        else:
            return 0
    elif i == len(linspace)-1:
        if x >= linspace[i-1] and x <= linspace[i]:
            return (x - linspace[i-1])/h[i-1]
        else:
            return 0
    else:
        if x >= linspace[i-1] and x <= linspace[i]:
            return (x - linspace[i-1])/h[i-1]
        elif x >= linspace[i] and x <= linspace[i+1]:
            return (linspace[i+1] - x)/h[i]
        else:
            return 0

In [ ]:
def func(x):
    return alpha + x + (beta - alpha - 1)*(np.exp(x/epsilon)-1)/(np.exp(1/epsilon)-1)

# Problem 1

In [ ]:
a = 0
b = 1
alpha = 2
beta = 4
epsilon = .02
N = 100

x_space = np.linspace(a,b,N+1)

matrix = create_matrix(x_space, alpha, beta, epsilon)

sol = spsolve(matrix[0], matrix[1])
y = [np.sum(sol[i]*phi(x,i,x_space) for i in range(N+1)) for x in x_space]

plt.plot(x_space, y, 'k')
plt.plot(x_space, func(x_space), 'g')
plt.show()

# Problem 2

In [ ]:
a = 0
b = 1
alpha = 2
beta = 4
epsilon = .02

even_grid = np.linspace(a,b,15)
clustered_grid = np.linspace(a,b,15)**(1/8)

even = create_matrix(even_grid, alpha, beta, epsilon)
clustered = create_matrix(clustered_grid, alpha, beta, epsilon)

sol_even = spsolve(even[0], even[1])
y_even = [np.sum(sol_even[i]*phi(x,i,even_grid) for i in range(15)) for x in even_grid]

sol_clustered = spsolve(clustered[0], clustered[1])
y_clustered = [np.sum(sol_clustered[i]*phi(x,i,clustered_grid) for i in range(15)) for x in clustered_grid]

plt.plot(even_grid, y_even, 'k')
plt.plot(clustered_grid, y_clustered, 'r')
plt.plot(even_grid, func(even_grid), 'g')
plt.show()

# Problem 3

In [ ]:
ns = []
errors = []
for i in range(4,22):
    X = np.linspace(a,b,2**i+1)
    ns.append(2**i)
    Actual = func(X)
    soln = create_matrix(X, alpha, beta, epsilon)
    soln = spsolve(soln[0], soln[1])
    error = np.max(np.abs(Actual-soln))
#     error = np.linalg.norm(Actual-soln)
    errors.append(error)
plt.loglog(ns, errors)
plt.xlabel('$n$', fontsize=16)
plt.ylabel('$E(n)$', fontsize=16)
#plt.savefig('FEM_error_2nd_order.pdf')
plt.show()
    